In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
import torch.nn.init as init
import time

### parameter ###
batchSize = 64
setEpoch = 300
### parameter ###

### dataset ###

#Set normalizer
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
#Set transform function
transform_train = transforms.Compose(
    [transforms.ToTensor(),
     normalize])
# transform_train = transforms.Compose(
#     [transforms.RandomCrop(32),
#      transforms.RandomHorizontalFlip(),
#      transforms.ToTensor(),
#      normalize])
transform_test = transforms.Compose(
    [transforms.ToTensor(),
     normalize])

#set dataset  
trainset = torchvision.datasets.CIFAR10(root='../data', train=True,
                                        download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR10(root='../data', train=False,
                                       download=True, transform=transform_test)
#set loader
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batchSize,
                                          shuffle=True, num_workers=2, pin_memory=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=batchSize,
                                         shuffle=False, num_workers=2,pin_memory=True)
#set class label on dataset
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

#import matplotlib.pyplot as plt
#import numpy as np


In [ ]:
#define NN

def _weights_init(m):
    classname = m.__class__.__name__
    #print(classname)
    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight)

class basicBlock(nn.Module):
    def __init__(self, inChannel, growthRate, layerDepth, dropRate = 0.2):
        super(basicBlock, self).__init__()
        self.layerDepth = layerDepth
        layers = []
        for i in range(layerDepth):
            layers.append(nn.BatchNorm2d(inChannel))
            # 1x1 conv
            layers.append(nn.Conv2d(inChannel, 4*growthRate, kernel_size=1, bias=False))
            layers.append(nn.BatchNorm2d(4*growthRate))
            # 3x3 conv
            layers.append(nn.Conv2d(4*growthRate, growthRate, kernel_size=3, padding = 1, bias=False))
            inChannel += growthRate
            
        self.moduleList = nn.ModuleList(layers)
        self.dropRate = dropRate
    def forward(self, x):
        for i in range(self.layerDepth):
            #BN, ReLU
            out = F.relu(self.moduleList[4*i](x))
            #1x1 conv (bottleNeck)
            out = self.moduleList[4*i + 1](out)
            #BN, ReLU
            out = F.relu(self.moduleList[4*i + 2](out))
            #3x3 conv
            out = self.moduleList[4*i + 3](out)    
            #dropout
            out = F.dropout(out, p = self.dropRate , training=self.training)
            #concatation
            x = torch.cat((x,out),dim=1)
        return x

class upNet(nn.Conv2d):
    def __init__(self):
        
    def forward(self, scale, outchannel):
        
class RDB(nn.Module):
    def __init__(self, inChannel, growthRate, layerDepth):
        layer = []
        self.layerDepth = layerDepth
        outChannel = inChannel
        for i in range(layerDepth):
            # 3x3 conv
            layers.append(nn.Conv2d(inChannel, growthRate, kernel_size=3, padding = 1, bias=False))
            inChannel += growthRate
        self.rdbLayers = nn.ModuleList(layers)
        #local feature fusion
        self.conv1x1=nn.Conv2d(inChannel, outChannel)
    def forward(self, x)
        localRes = x
        for i in range(self.layerDepth):
            out = self.moduleList[i](x)
            out = nn.relu(out)
            x = torch.cat((x,out),dim=1)
        
        x = localRes + x
        return x
    
class RDN(nn.Module):

    def __init__(self):
        super(denseNet, self).__init__()
        #parameter
        self.inChannel = 24
        self.growthRate = 64
        self.blockDepth = 16
        self.convDepth = 8
        
        layers = []
        for i in range(self.blockDepth):
            layers.append = RDB(inChannel, growthRate, convDepth)
        self.rdbs = nn.ModuleList(layers)
            
#         self.conv0 = nn.Conv2d(3, self.inChannel , kernel_size=3, padding = 1, bias = False)
        
#         #transicion layer
#         TLdepth1 = self.inChannel + (self.growthRate)*self.layerDepth
#         self.TLbn1 = (nn.BatchNorm2d(TLdepth1))
#         self.convTL1  = nn.Conv2d(TLdepth1, self._theta(TLdepth1), kernel_size = 1, bias = False)
#         self.avgPool1 = nn.AvgPool2d(2, stride = 2)
        
#         TLdepth2 = self._theta(TLdepth1) + (self.growthRate)*self.layerDepth
#         self.TLbn2 = (nn.BatchNorm2d(TLdepth2))
#         self.convTL2  = nn.Conv2d(TLdepth2, self._theta(TLdepth2), kernel_size = 1, bias = False)
#         self.avgPool2 = nn.AvgPool2d(2, stride = 2)
        
#         #dense blk
#         #when cifar, layer's depth are same
#         self.dense1 = basicBlock(self.inChannel, self.growthRate, self.layerDepth)
#         self.dense2 = basicBlock(self._theta(TLdepth1), self.growthRate, self.layerDepth)
#         self.dense3 =  basicBlock(self._theta(TLdepth2), self.growthRate, self.layerDepth)
        
#         #classification Layer
#         depthClass = self._theta(TLdepth2) +  (self.growthRate)*self.layerDepth
#         self.GlobalAvgPool = nn.AvgPool2d(8, stride = 1)
#         self.linear = nn.Linear(depthClass, 10)
        
        
#         self.apply(_weights_init)

    
    def forward(self, x):
        #SFENet
        x = self.sfe1(x)
        out = self.sfe2(x)
        #RDBs
        out = self.rdbs(out)
        #DFF
        out = self.gff(out)
        x = x + out
        #UPNet
        
        
        return x